<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/Fine_Tunning_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-r3fma_vi/unsloth_339d5d5978e645759252d013207a6bdb
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-r3fma_vi/unsloth_339d5d5978e645759252d013207a6bdb
  Resolved https://github.com/unslothai/unsloth.git to commit e5381b6a263dce315ed4d44da1dd4f188d6800a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade bitsandbytes
!pip install --upgrade trl
!pip install --upgrade datasets


  Using cached trl-0.11.0-py3-none-any.whl.metadata (12 kB)
Using cached trl-0.11.0-py3-none-any.whl (316 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6


In [ ]:
!pip install triton

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 55 Mega/titles_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 55 Mega/formated_titles_dataset_chat_data.json"
max_seq_length = 2028
model_name = "unsloth/llama-3-8b-bnb-4bit"
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|title|]") + len("[|title|]")
        news_end = full_text.find("[|etitle|]")
        summary_start = full_text.find("[|content|]") + len("[|content|]")
        summary_end = full_text.find("[|econtent|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    # Iterate over the list of dictionaries
    try: # this will allow the program to continue if there is a KeyboardInterrupt
        for item in data:
            # Access the 'input' key for each dictionary
            for prompt in item['input']:
                instruction, input_text, response = separate_text(prompt)
                instructions.append(instruction)
                inputs.append(input_text)
                outputs.append(response)
    except KeyboardInterrupt:
        pass # this will ignore the KeyboardInterrupt and allow the program to continue

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON

    with open(OUTPUT_PATH_DATASET, 'w') as output_file:

        json.dump(formatted_data, output_file, indent=4)

In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
from transformers import AutoModelForCausalLM

# Carregar o modelo com offloading para CPU e 8-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    device_map="auto",  # Mapear automaticamente partes do modelo para CPU e GPU

)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer # Make sure to import AutoTokenizer

# Carregar o modelo pré-treinado sem o fine-tuning

model_name = "unsloth/llama-3-8b-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)



# Definir os prompts de teste

test_prompts = [

    "Explique o que é aprendizado supervisionado.",

    "Resuma a história da Segunda Guerra Mundial.",

    "Quais são as principais vantagens de um algoritmo de árvore de decisão?"

]



# Gerar respostas para os prompts de teste antes do fine-tuning

for prompt in test_prompts:

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Enviar para a GPU se disponível

    outputs = model.generate(**inputs, max_new_tokens=128)

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Prompt: {prompt}\nResposta: {decoded_output}\n")



# Salvar os resultados antes do fine-tuning para comparação futura

with open("test_results_before_finetuning.txt", "w") as f:

    for prompt in test_prompts:

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens=128)

        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        f.write(f"Prompt: {prompt}\nResposta: {decoded_output}\n\n")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Prompt: Explique o que é aprendizado supervisionado.
Resposta: Explique o que é aprendizado supervisionado. Explique o que é aprendizado não supervisionado. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem supervisão. Explique o que é aprendizado sem superv

Prompt: Resuma a história da Segunda Guerra Mundial.
Resposta: Resuma a história da Segunda Guerra Mundial. A história da Segunda Guerra Mundial se inicia no dia 1º de setembro de 1939, quando o exército alemão invadiu a Polônia. A Alemanha, que já havia anexado a Áustria, o Sudetenland, a Checoslováquia e a Polônia, se preparava para a guerra. Os Aliados, formados pela Inglaterra, a França, a 

In [ ]:
# Load the dataset
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

format_dataset_into_model_input(data)

# Set load_in_8bit_fp32_cpu_offload to True to allow CPU and disk offloading
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto",
    load_in_8bit_fp32_cpu_offload=True # add this argument
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 55 Mega/formated_titles_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/48998 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/48998 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 48,998 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.388000
2,2.384800
3,2.716000
4,2.510800
5,2.250800
6,2.161500
7,2.095700
8,1.959900
9,1.912300
10,2.121300


In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS CONTENT",
        "Explique o que é aprendizado supervisionado.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSUMMARIZE THIS CONTENT\n\n### Input:\nExplique o que é aprendizado supervisionado.\n\n### Response:\nThis book is intended to provide a thorough introduction to the field of supervised learning, with a focus on both the theoretical and practical aspects of the field. The book is divided into two parts. Part I presents the basic concepts of supervised learning. Part II discusses more advanced concepts and more recent developments in the field. The book']

In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS CONTENT",
        "Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison. With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end. The introduction discusses not so much this project as the past and present social and political developments in this young nation, placing the images firmly in context. And indeed, throughout the coverage of agriculture, sheep, beaches, and apartment houses are the ubiquitous soldiers with guns. The heat and aridity are everywhere evident on this day (May 10, 1984) in the ``green'' month, before things are parched to brown. Both books use a chronological arrangement and tell when, to the hour, the pictures were taken, but Canada , being so large, offers maps on each page to show the geographical location of particular images. Both recommended for their candid presentation of ordinary daily life. Kathleen Collins, Library of CongressCopyright 1985 Reed Business Information, Inc.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS CONTENT

### Input:
Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis'mostly native 55), to produce 100,000 images (to Israelis'40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somb

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Desafio_Fase_3/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Desafio_Fase_3/lora_model")


('/content/drive/MyDrive/Desafio_Fase_3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Desafio_Fase_3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Desafio_Fase_3/lora_model/tokenizer.json')